<a href="https://colab.research.google.com/github/ayush5/-CS69002_9A_18CS60R26/blob/master/MLP_Tutorial_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

## Import Header files

In [0]:
import torch
import pandas as pd
import numpy as np

In [0]:
print('Welcome to Computing Lab 2. The current PyTorch version is ', torch.__version__)

Welcome to Computing Lab 2. The current PyTorch version is  1.0.1.post2


## Load the dataset and visualize


There are 3 ways to do this:

(a)  From Github (Files < 25MB)

The easiest way to upload a CSV file is from your GitHub repository. Click on the dataset in your repository, then click on View Raw. Copy the link to the raw dataset and store it as a string variable called url in Colab as shown below (a cleaner method but it’s not necessary). The last step is to load the url into Pandas read_csv to get the dataframe.


```
url = 'copied_raw_GH_link'
df1 = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

```

---


(b) From a local drive

To upload from your local drive, start with the following code:

```
from google.colab import files
uploaded = files.upload()
```

It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.

Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).
```
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe
```

---


(c) From Google drive

This the most complicated of the three methods. Follow this [link](https://medium.freecodecamp.org/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa) to learn more.

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df_train = pd.read_csv(url, sep='\t')
url="https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv"
df_test= pd.read_csv(url, sep="\t")

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['check.csv'])

In [0]:
import io
df = pd.read_csv(io.StringIO(uploaded['check.csv'].decode('utf-8')), sep='\t')
df.head()

## Pandas

[Pandas](https://pandas.pydata.org/) is a high-performance, easy-to-use data structure and data analytics tool for Python.

Tutorials:

*   [Link 1](https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html)
*   [Link 2](https://www.tutorialspoint.com/python_pandas)
*   [Link 3](https://www.dataquest.io/blog/pandas-python-tutorial/)



In [0]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

NameError: ignored

## Data pre-processing
The first step when building a neural network model is getting your data into the proper form to feed into the network. We'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can use any type of pre-processing that you see fit, based on the task. For the tutorial, i will show just one pre-processing step: **lowercasing**.

In [3]:
text_reviews = df_train['text'].astype(str).tolist()
text_labels = df_train['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
text_reviews[0], text_labels[0]

('john waters has given us a genuinely enjoyable film. this certainly isn\'t without its shocking waters-esque moments, but it is tamer than his older culty stuff, such as "pink flamingoes". "pecker" harkens back to john\'s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "polyester". overall, a really fun comedy with some great moments!',
 1)

In [0]:
len(text_reviews)

17999

In [0]:
text_reviews = df_test['text'].astype(str).tolist()
text_labels = df_test['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
text_reviews[0], text_labels[0]

("as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i wa

In [0]:
len(test_reviews)

NameError: ignored

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  for sent in dataset:
      for word in sent.split():
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
  return word_to_ix

In [0]:
generate_word_ids(['welcome to computing lab 2'])

{'2': 4, 'computing': 2, 'lab': 3, 'to': 1, 'welcome': 0}

In [47]:
train_text= df_train['text'].astype(str).tolist()
train_label=df_train['label'].astype(int)
test_text=df_test['text'].astype(str).tolist()
test_label=df_test['label'].astype(int)
print (train_text[0])

John Waters has given us a genuinely enjoyable film. This certainly isn't without its shocking Waters-esque moments, but it is tamer than his older culty stuff, such as "Pink Flamingoes". "Pecker" harkens back to John's early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "Polyester". Overall, a really fun comedy with some great moments!


In [165]:
from string import punctuation
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
  
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [166]:
train_text= df_train['text'].astype(str).tolist()
train_label=df_train['label'].astype(int)
test_text=df_test['text'].astype(str).tolist()
test_label=df_test['label'].astype(int)

train_reviews_cleaned = []
for i in range(len(train_text)):
    train_reviews_cleaned.append(''.join([c.lower() for c in train_text[i] if c not in punctuation and c.isdigit()==False]))
train_text=train_reviews_cleaned
train_reviews_cleaned=[]
for line in train_text:
  words=line.split()
  sentence=[]
  for word in words:
    if word not in stop_words and word.isdigit()==False:
      word_n=(lemmatizer.lemmatize(word,"v"))
      word_n=(lemmatizer.lemmatize(word_n,"r"))
      word_n=(lemmatizer.lemmatize(word_n,"a"))
      sentence.append(lemmatizer.lemmatize(word_n))
  n_sentence=""
  for word in sentence:
    n_sentence+=word+" "
  train_reviews_cleaned.append(n_sentence)
train_text=train_reviews_cleaned  


test_reviews_cleaned = []
for i in range(len(test_text)):
    test_reviews_cleaned.append(''.join([c.lower() for c in test_text[i] if c not in punctuation and c.isdigit()==False]))
test_text=test_reviews_cleaned
test_reviews_cleaned=[]
for line in test_text:
  words=line.split()
  sentence=[]
  for word in words:
    if word not in stop_words and word.isdigit()==False:
      word_n=(lemmatizer.lemmatize(word,"v"))
      word_n=(lemmatizer.lemmatize(word_n,"r"))
      word_n=(lemmatizer.lemmatize(word_n,"a"))
      sentence.append(lemmatizer.lemmatize(word_n))
  n_sentence=""
  for word in sentence:
    n_sentence+=word+" "
  test_reviews_cleaned.append(n_sentence)
test_text=test_reviews_cleaned
print (train_text[0])
print (test_text[0])


john water give u genuinely enjoyable film certainly isnt without shock watersesque moment tame old culty stuff pink flamingo pecker harken back john early mainstream stage remind viewer kind humor evident polyester overall really fun comedy great moment 
always inaccurate picture homeless tv tell lot lie panhandler early make everyone look bad claim make day day much close reality someone drive hold sign offer work offer work actually go take work physically ableand would offer id damn sure invest apt prepay least year keep bank still leave nl cash game casino usually always win could win decent bankroll instead win month play always minimum buy due want risk lose homeless cause didnt wanna risk spend money go break sometimes sock sleep outside anyone want talk contact sevencard yahoo messengeri admit different homeless people though due fact never drink smoke take drug im long homeless govt house month get ssi spend time win online poker mom sunflower diversify work hard get ssi glad

In [0]:

train_text=train_reviews_cleaned

In [0]:
#data = [("me gusta comer en la cafeteria", "SPANISH"),
        ("Give it to me", "ENGLISH"),
        ("No creo que sea una buena idea", "SPANISH"),
        ("No it is not a good idea to get lost at sea", "ENGLISH")]

#test_data = [("Yo creo que si", "SPANISH"),
             ("it is lost on me", "ENGLISH")]

In [167]:
word_to_ix = generate_word_ids(train_text+test_text)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

96021


In [0]:
from collections import Counter
def vocabulary_reduction(reviews,labels,min_freq,polarity_cut_off):
    
    pos_count = Counter()
    neg_count = Counter()
    tot_count = Counter()

    for i in range(len(reviews)):
        for word in reviews[i].split():
            tot_count[word]+=1
            if labels[i] == 1:
                pos_count[word]+=1
            else:
                neg_count[word]+=1    
    
    #Identify words with frequency greater than min_freq
    vocab_freq = []
    print(min_freq)
    for word in tot_count.keys():
        if tot_count[word]>min_freq:
            vocab_freq.append(word) 
    
    
    #Use polarity to reduce vocab
    pos_neg_ratio = Counter()
    vocab_pos_neg = (set(pos_count.keys())).intersection(set(neg_count.keys()))
    for word in vocab_pos_neg:
        if tot_count[word]>60:
            ratio = pos_count[word]/float(neg_count[word]+1)
            if ratio>1:
                pos_neg_ratio[word] = np.log(ratio)
            else:
                pos_neg_ratio[word] = -np.log(1/(ratio+0.01)) 
    
    mean_ratio = np.mean(list(pos_neg_ratio.values()))
    
    vocab_polarity = []
    for word in pos_neg_ratio.keys():
        if (pos_neg_ratio[word]<(mean_ratio-polarity_cut_off)) or (pos_neg_ratio[word]>(mean_ratio+polarity_cut_off)):
            vocab_polarity.append(word)
     
    vocab_rm_polarity = set(pos_neg_ratio.keys()).difference(vocab_polarity)
    vocab_reduced = (set(vocab_freq)).difference(set(vocab_rm_polarity))
 
    reviews_cleaned = []
    
    for review in reviews:
        review_temp = [word for word in review.split() if word in vocab_reduced]
        reviews_cleaned.append(' '.join(review_temp))
    
    return reviews_cleaned

In [169]:
train_text=vocabulary_reduction(train_text,train_label,3,0.01)

3


In [170]:
word_to_ix = generate_word_ids(train_text)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

23414


## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

lin = nn.Linear(5,3)
data1 = Variable(torch.randn(10,5)) # create a 10 * 5 matrix of random vars

In [0]:
data1, data1.size()

(tensor([[-0.4974,  0.4396, -0.7581,  1.0783,  0.8008],
         [ 1.6806,  0.3559, -0.6866,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599, -0.3097],
         [-0.3957, -0.2234,  1.7174,  0.3189, -0.4245],
         [ 0.3057, -0.7746,  0.0349,  0.3211, -0.8798],
         [-0.6011, -1.2742,  2.1228, -1.2347, -0.4879],
         [-1.4181,  0.8963,  0.0780,  0.5258,  0.3466],
         [-0.1973, -1.0546,  1.2780,  0.1453,  0.2311],
         [ 0.0087, -0.1423,  0.5750, -0.6417, -2.2064],
         [-0.7508,  2.8140,  0.3598, -0.0898,  0.4584]]), torch.Size([10, 5]))

In [0]:
lin(data1), lin(data1).size()

(tensor([[ 0.3739,  0.1674, -0.1029],
         [ 0.8355,  0.0414,  0.8471],
         [ 0.2831,  0.6893, -0.1183],
         [-0.2887,  1.0737,  0.3888],
         [-0.0317,  0.9654, -0.0386],
         [-1.4234,  0.7988,  0.2472],
         [-0.0414,  0.1357, -0.2777],
         [-0.6189,  0.8737,  0.3820],
         [-0.2207,  0.9984, -0.2671],
         [ 0.6052, -0.4270,  0.2221]], grad_fn=<AddmmBackward>),
 torch.Size([10, 3]))

In [0]:
lin.weight.size()

torch.Size([3, 5])

**BE VERY CAREFUL WITH THE DIMENSIONS**

In [172]:
F.relu(lin(data1))  # Activation Function

tensor([[0.3739, 0.1674, 0.0000],
        [0.8355, 0.0414, 0.8471],
        [0.2831, 0.6893, 0.0000],
        [0.0000, 1.0737, 0.3888],
        [0.0000, 0.9654, 0.0000],
        [0.0000, 0.7988, 0.2472],
        [0.0000, 0.1357, 0.0000],
        [0.0000, 0.8737, 0.3820],
        [0.0000, 0.9984, 0.0000],
        [0.6052, 0.0000, 0.2221]], grad_fn=<ReluBackward0>)

## Model Definition for the BOWClassifier

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
    
  def forward(self, x):
    z=F.softmax(self.lin(x))
    return F.softmax(self.lin2(z))

##Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            continue
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()

In [190]:
for param in bow.parameters():
    print(param,param.size())

Parameter containing:
tensor([[ 0.0007,  0.0023, -0.0009,  ..., -0.0011,  0.0030, -0.0052],
        [ 0.0017, -0.0046,  0.0063,  ..., -0.0021,  0.0055,  0.0026],
        [-0.0033,  0.0058,  0.0050,  ..., -0.0011, -0.0032,  0.0007],
        ...,
        [-0.0047,  0.0011,  0.0032,  ...,  0.0008, -0.0057, -0.0020],
        [ 0.0049,  0.0044, -0.0018,  ..., -0.0003,  0.0020, -0.0035],
        [ 0.0016,  0.0006, -0.0058,  ...,  0.0025, -0.0043,  0.0044]],
       device='cuda:0', requires_grad=True) torch.Size([100, 23414])
Parameter containing:
tensor([-1.2063e-03, -4.7912e-03, -1.9344e-03,  2.9490e-03,  3.8029e-03,
        -5.6944e-03, -1.5579e-03, -3.1249e-03,  7.6899e-04, -5.8828e-03,
        -4.9178e-03,  5.1587e-03, -4.1779e-03, -9.6015e-04,  4.4293e-03,
        -4.2960e-03, -3.9731e-03, -5.7628e-03,  2.3381e-03, -1.0922e-04,
        -1.8987e-03, -2.3788e-03, -1.1575e-03,  3.9075e-03,  3.5598e-03,
        -4.3354e-03,  1.6433e-03, -5.3862e-03, -5.9353e-03,  1.5955e-03,
         1.0995

### Let's check with a sample case how our model is working

In [191]:
label_to_ix = {0: 0, 1: 1}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [160]:
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>15:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
      #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
      #print('prediction: {}'.format(ix_to_label[pred]))
      #print('actual: {}'.format(label))
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0.8552
17
0.8526
18
0.8528
19
0.8594


In [0]:
total_count=0
correct_count=0
print('--- AFTER TRAINING ---')
for y,instance in enumerate(test_text):
    label=test_label[y]
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.cpu().numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    print('actual: {}'.format(label))
    if label==ix_to_label[pred]:
      correct_count+=1
    total_count+=1
    print (correct_count/total_count)

--- AFTER TRAINING ---
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[9.9991e-01, 8.5448e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[9.9991e-01, 8.5448e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[9.9991e-01, 8.5448e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[8.7341e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


tensor([[9.9991e-01, 8.5448e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[9.9991e-01, 8.5448e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[9.9991e-01, 8.5451e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[8.7336e-05, 9.9991e-01]], device='cuda:0', grad_fn=<SoftmaxBackw

In [0]:
print (correct_count/total_count)

0.8078


In [161]:
torch.save(bow,'model_1a.json')

from google.colab import files
files.download("model_1a.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


KeyboardInterrupt: ignored

In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
torch.load(io.BytesIO(temp_test['model.json']))

TASK 1B

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, num_labels)
    
    
  def forward(self, x):
    z=F.softmax(self.lin(x))
    return F.softmax(self.lin2(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [195]:
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>17:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
      #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
      #print('prediction: {}'.format(ix_to_label[pred]))
      #print('actual: {}'.format(label))
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0.8524
19
0.8496


In [196]:
torch.save(bow,'model_1b.json')

from google.colab import files
files.download("model_1b.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Task 1c

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin2 = nn.Linear(200, num_labels)
    
    
  def forward(self, x):
    z=F.softmax(self.lin(x))
    return F.softmax(self.lin2(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [201]:
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>17:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
      #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
      #print('prediction: {}'.format(ix_to_label[pred]))
      #print('actual: {}'.format(label))
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0.8348
19
0.8506


In [199]:
torch.save(bow,'model_1c.json')

from google.colab import files
files.download("model_1c.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**TASK 1D**

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 500)
    self.lin2 = nn.Linear(500, num_labels)
    
    
  def forward(self, x):
    z=(self.lin(x))
    return F.softmax(self.lin2(z),dim=1)
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [236]:
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>17:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
      #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
      #print('prediction: {}'.format(ix_to_label[pred]))
      #print('actual: {}'.format(label))
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0.8436
19
0.8558


In [206]:
torch.save(bow,'model_1d.json')

from google.colab import files
files.download("model_1d.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
from sklearn.metrics import classification_report

TASK 1E

In [208]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 500)
    self.lin2 = nn.Linear(500, num_labels)
    
    
  def forward(self, x):
    z=F.softmax(self.lin(x))
    return F.softmax(self.lin2(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      

torch.save(bow,'model_1d.json')
from google.colab import files
files.download("model_1d.json")

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.8394


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


MessageError: ignored

In [209]:
print(classification_report(true_label,pred_label))

              precision    recall  f1-score   support

           0       0.90      0.76      0.82      2482
           1       0.79      0.92      0.85      2518

   micro avg       0.84      0.84      0.84      5000
   macro avg       0.85      0.84      0.84      5000
weighted avg       0.85      0.84      0.84      5000



In [211]:
torch.save(bow,'model_1d.json')

from google.colab import files
files.download("model_1d.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


KeyboardInterrupt: ignored

**2A**

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 10)
    self.lin2 = nn.Linear(10, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=torch.tanh(self.lin(x))
    z=torch.tanh(self.lin2(z1))
    return F.softmax(self.lin3(z),dim=1)
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>=0:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [217]:
torch.save(bow,'model_2a.json')
from google.colab import files
files.download("model_2a.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [218]:
print(classification_report(true_label,pred_label))

              precision    recall  f1-score   support

           0       0.94      0.41      0.57      2482
           1       0.63      0.97      0.76      2518

   micro avg       0.69      0.69      0.69      5000
   macro avg       0.78      0.69      0.67      5000
weighted avg       0.78      0.69      0.67      5000



2B

In [220]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=(self.lin(x))
    z=(self.lin2(z1))
    return F.softmax(self.lin3(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(10):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch>=0:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


0.5036
1
0.5036
2
0.7454
3
0.7724
4
0.6722
5
0.7684
6
0.7894
7
0.805
8
0.748
9
0.7346
10
0.7928
11
0.7832
12
0.769
13
0.8054
14
0.7998
15
0.783
16
0.804
17
0.7732
18
0.8012
19
0.7456


In [221]:
torch.save(bow,'model_2b.json')
from google.colab import files
files.download("model_2b.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [222]:
print(classification_report(true_label,pred_label))

              precision    recall  f1-score   support

           0       0.78      0.67      0.72     49640
           1       0.72      0.81      0.76     50360

   micro avg       0.74      0.74      0.74    100000
   macro avg       0.75      0.74      0.74    100000
weighted avg       0.75      0.74      0.74    100000



2C

In [232]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin2 = nn.Linear(30, 30)
    self.lin3 = nn.Linear(30, num_labels)
    
    
  def forward(self, x):
    z1=self.lin(x)
    z=self.lin2(z1)
    return F.softmax(self.lin3(z), dim = 1)
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.05) 

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



0
0.814
1


KeyboardInterrupt: ignored

In [224]:
torch.save(bow,'Task2c.mdl')
from google.colab import files
files.download("Task2c.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [225]:
print(classification_report(true_label,pred_label))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2482
           1       0.50      1.00      0.67      2518

   micro avg       0.50      0.50      0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.34      5000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2D

In [226]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, 50)
    self.lin3 = nn.Linear(50, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z=F.softmax(self.lin2(z1))
    return F.softmax(self.lin3(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


0
1
2
3


KeyboardInterrupt: ignored

In [0]:
torch.save(bow,'Task2d.mdl')
from google.colab import files
files.download("Task2d.mdl")

In [0]:
print(classification_report(true_label,pred_label))

2E

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, 50)
    self.lin3 = nn.Linear(50, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z=F.softmax(self.lin2(z1))
    return F.softmax(self.lin3(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task2e.mdl')
from google.colab import files
files.download("Task2e.mdl")

In [0]:
print(classification_report(true_label,pred_label))

3A

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, 50)
    self.lin3 = nn.Linear(50, 10)
    self.lin4 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z2=F.softmax(self.lin2(z1))
    z=F.softmax(self.lin3(z2))
    return F.softmax(self.lin4(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task3a.mdl')
from google.colab import files
files.download("Task3a.mdl")

In [0]:
print(classification_report(true_label,pred_label))

3b

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin2 = nn.Linear(200, 100)
    self.lin3 = nn.Linear(100, 10)
    self.lin4 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z2=F.softmax(self.lin2(z1))
    z=F.softmax(self.lin3(z2))
    return F.softmax(self.lin4(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task3b.mdl')
from google.colab import files
files.download("Task3b.mdl")

In [0]:
print(classification_report(true_label,pred_label))

4a

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin2 = nn.Linear(30, 20)
    self.lin3 = nn.Linear(20, 10)
    slef.dp = nn.Dropout(p=0.3)
    self.lin4 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z1=self.dp(z1)/0.3
    z2=F.softmax(self.lin2(z1))
    z2=self.dp(z2)/0.3
    z=F.softmax(self.lin3(z2))
    z=self.dp(z)
    return F.softmax(self.lin4(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task4a.mdl')
from google.colab import files
files.download("Task4a.mdl")

In [0]:
print(classification_report(true_label,pred_label))

4b

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, 100)
    slef.dp = nn.Dropout(p=0.3)
    self.lin3 = nn.Linear(100, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z1=self.dp(z1)/0.5
    z=F.softmax(self.lin2(z1))
    z=self.dp(z)/0.5
    return F.softmax(self.lin3(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task4b.mdl')
from google.colab import files
files.download("Task4b.mdl")

In [0]:
print(classification_report(true_label,pred_label))

4C

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, 10)
    slef.dp = nn.Dropout(p=0.3)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z1=self.dp(z1)/0.5
    z=F.softmax(self.lin2(z1))
    z=self.dp(z)/0.5
    return F.softmax(self.lin3(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task4c.mdl')
from google.colab import files
files.download("Task4c.mdl")

In [0]:
print(classification_report(true_label,pred_label))

5a

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin2 = nn.Linear(30, 20)
    self.lin3 = nn.Linear(20, 10)
    slef.dp = nn.Dropout(p=0.3)
    self.lin4 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    z1=F.softmax(self.lin(x))
    z1=self.dp(z1)/0.3
    z2=F.softmax(self.lin2(z1))
    z2=self.dp(z2)/0.3
    z=F.softmax(self.lin3(z2))
    z=self.dp(z)
    return F.softmax(self.lin4(z))
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

true_label=[]
pred_label=[]
# the training loop
for epoch in range(20):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label, label_to_ix)).cuda()
        probs = bow(bow_vec) # forward pass
        probs = probs
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
    total_count=0
    correct_count=0
    #print('--- AFTER TRAINING ---')
    if epoch==19:
      for y,instance in enumerate(test_text):
        label=test_label[y]
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        logprobs = bow(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        #print('prediction: {}'.format(ix_to_label[pred]))
        #print('actual: {}'.format(label))
        true_label.append(label)
        pred_label.append(ix_to_label[pred])
        if label==ix_to_label[pred]:
          correct_count+=1
        total_count+=1
      print (correct_count/total_count)
      



In [0]:
torch.save(bow,'Task4c.mdl')
from google.colab import files
files.download("Task4c.mdl")

In [0]:
print(classification_report(true_label,pred_label))